# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data

School ID            school_name      type  size   budget
0           0      Huang High School  District  2917  1910635
1           1   Figueroa High School  District  2949  1884411
2           2    Shelton High School   Charter  1761  1056600
3           3  Hernandez High School  District  4635  3022020
4           4    Griffin High School   Charter  1468   917500
5           5     Wilson High School   Charter  2283  1319574
6           6    Cabrera High School   Charter  1858  1081356
7           7     Bailey High School  District  4976  3124928
8           8     Holden High School   Charter   427   248087
9           9       Pena High School   Charter   962   585858
10         10     Wright High School   Charter  1800  1049400
11         11  Rodriguez High School  District  3999  2547363
12         12    Johnson High School  District  4761  3094650
13         13       Ford High School  District  2739  1763916
14         14     Thomas High School   Charter  1635  1043130

In [2]:
student_data

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  
0                 66          79  
1                 94          61  
2                 90          60  
3                 67          58  
4                 97          84  
...              ...         ...  
39165             99          90  
39166             95          70  
39167             73          84  
39168             99          90  
39169             95          75  

[39170 rows x 7 columns]

In [3]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  School ID      type  size   budget  
0             66          79          0  District  2917  1910635  
1             94          61          0  District  2917  1910635  
2             90          60          0  District  2917  1910635  
3             67          58          0  District  2917  1910635  
4             97          84          0  District  2917  1910635

## District Summary

In [4]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["School ID"].unique())
school_count

15

In [5]:
# Calculate the total number of students
student_count = len(school_data_complete["Student ID"].unique())
student_count

39170

In [6]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [7]:
# Calculate the average (mean) math score
average_math_score = student_data["math_score"].mean()
average_math_score

78.98537145774827

In [8]:
# Calculate the average (mean) reading score
average_reading_score = student_data["reading_score"].mean()
average_reading_score

81.87784018381414

In [9]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [10]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = (average_reading_score + average_reading_score)/2
print (f"Passing reading average percentage: {passing_reading_count}%")

Passing reading average percentage: 81.87784018381414%


In [11]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
print (f"Overall passing average percentage: {overall_passing_rate}%")

Overall passing average percentage: 65.17232575950983%


In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame ????????????????????????????
district_results = [{"Total Schools": school_count, 
            "Total Students": student_count, 
            "Total Budget": total_budget, 
            "Average Math Score":  round(average_math_score,2), 
            "Average Reading Score":  round(average_reading_score,2), 
            "% Passing Math": round(passing_math_percentage,2),
            "% Passing Reading": round(passing_reading_count,2),
            "% Overall Passing Rate": round(overall_passing_rate,2)}]

district_summary = pd.DataFrame(district_results)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary

Total Schools Total Students    Total Budget  Average Math Score  \
0             15         39,170  $24,649,428.00               78.99   

   Average Reading Score  % Passing Math  % Passing Reading  \
0                  81.88           74.98              81.88   

   % Overall Passing Rate  
0                   65.17

## School Summary

In [13]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [14]:
# Use the code provided to select the school type
school_data_complete["passing_math"] = school_data_complete["math_score"] >= 70
school_data_complete["passing_reading"] = school_data_complete["reading_score"] >= 70

school_data_complete

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  \
0                 66          79          0  District  2917  1910635   
1                 94          61          0  District  2917  1910635   
2                 90          60          0  District  2917  1910635   
3                 67          58          0  District  2917  1910635   
4                 97          84          0  District  2917  1910635   
...              ...         ...        ...       ...   ...      ...   
39165             99          90         14   Charter  1635  1043130   
39166             95          70         14   Charter  1635  1043130   
39167             73          84         14   Charter  1635  1043130   
39168             99          90         14   Charter  1635  1043130   
39169             95          75         14   Charter  1635  1043130   

       passing_math  passing_reading  
0              True            False  
1             False             True  
2             False             True  
3             False            False  
4              True             True  
...             ...              ...  
39165          True             True  
39166          True             True  
39167          True             True  
39168          True             True  
39169          True             True  

[39170 rows x 13 columns]

In [15]:
# Calculate the total student count
per_school_counts = school_data_complete.groupby("school_name").size()
per_school_counts

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
dtype: int64

In [16]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_budget

school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
Name: budget, dtype: float64

In [17]:
per_school_capita = per_school_budget / per_school_counts
per_school_capita

school_name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [18]:
# Calculate the average test scores - math
per_school_math = school_data_complete["math_score"].mean()
per_school_math

78.98537145774827

In [19]:
# Calculate the average test scores - reading
per_school_reading = school_data_complete["reading_score"].mean()
per_school_reading

81.87784018381414

In [20]:
# Calculate the number of schools with math scores of 70 or higher - math
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
school_passing_math

Student ID     student_name gender grade         school_name  \
0               0     Paul Bradley      M   9th   Huang High School   
4               4       Bonnie Ray      F   9th   Huang High School   
5               5    Bryan Miranda      M   9th   Huang High School   
6               6    Sheena Carter      F  11th   Huang High School   
8               8     Michael Roth      M  10th   Huang High School   
...           ...              ...    ...   ...                 ...   
39165       39165     Donna Howard      F  12th  Thomas High School   
39166       39166        Dawn Bell      F  10th  Thomas High School   
39167       39167   Rebecca Tanner      F   9th  Thomas High School   
39168       39168     Desiree Kidd      F  10th  Thomas High School   
39169       39169  Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  \
0                 66          79          0  District  2917  1910635   
4                 97          84          0  District  2917  1910635   
5                 94          94          0  District  2917  1910635   
6                 82          80          0  District  2917  1910635   
8                 95          87          0  District  2917  1910635   
...              ...         ...        ...       ...   ...      ...   
39165             99          90         14   Charter  1635  1043130   
39166             95          70         14   Charter  1635  1043130   
39167             73          84         14   Charter  1635  1043130   
39168             99          90         14   Charter  1635  1043130   
39169             95          75         14   Charter  1635  1043130   

       passing_math  passing_reading  
0              True            False  
4              True             True  
5              True             True  
6              True             True  
8              True             True  
...             ...              ...  
39165          True             True  
39166          True             True  
39167          True             True  
39168          True             True  
39169          True             True  

[29370 rows x 13 columns]

In [21]:
# Calculate the number of schools with reading scores of 70 or higher - reading
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
school_passing_reading

Student ID     student_name gender grade         school_name  \
1               1     Victor Smith      M  12th   Huang High School   
2               2  Kevin Rodriguez      M  12th   Huang High School   
4               4       Bonnie Ray      F   9th   Huang High School   
5               5    Bryan Miranda      M   9th   Huang High School   
6               6    Sheena Carter      F  11th   Huang High School   
...           ...              ...    ...   ...                 ...   
39165       39165     Donna Howard      F  12th  Thomas High School   
39166       39166        Dawn Bell      F  10th  Thomas High School   
39167       39167   Rebecca Tanner      F   9th  Thomas High School   
39168       39168     Desiree Kidd      F  10th  Thomas High School   
39169       39169  Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  \
1                 94          61          0  District  2917  1910635   
2                 90          60          0  District  2917  1910635   
4                 97          84          0  District  2917  1910635   
5                 94          94          0  District  2917  1910635   
6                 82          80          0  District  2917  1910635   
...              ...         ...        ...       ...   ...      ...   
39165             99          90         14   Charter  1635  1043130   
39166             95          70         14   Charter  1635  1043130   
39167             73          84         14   Charter  1635  1043130   
39168             99          90         14   Charter  1635  1043130   
39169             95          75         14   Charter  1635  1043130   

       passing_math  passing_reading  
1             False             True  
2             False             True  
4              True             True  
5              True             True  
6              True             True  
...             ...              ...  
39165          True             True  
39166          True             True  
39167          True             True  
39168          True             True  
39169          True             True  

[33610 rows x 13 columns]

In [22]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]

passing_math_and_reading

Student ID     student_name gender grade         school_name  \
4               4       Bonnie Ray      F   9th   Huang High School   
5               5    Bryan Miranda      M   9th   Huang High School   
6               6    Sheena Carter      F  11th   Huang High School   
8               8     Michael Roth      M  10th   Huang High School   
9               9   Matthew Greene      M  10th   Huang High School   
...           ...              ...    ...   ...                 ...   
39165       39165     Donna Howard      F  12th  Thomas High School   
39166       39166        Dawn Bell      F  10th  Thomas High School   
39167       39167   Rebecca Tanner      F   9th  Thomas High School   
39168       39168     Desiree Kidd      F  10th  Thomas High School   
39169       39169  Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  \
4                 97          84          0  District  2917  1910635   
5                 94          94          0  District  2917  1910635   
6                 82          80          0  District  2917  1910635   
8                 95          87          0  District  2917  1910635   
9                 96          84          0  District  2917  1910635   
...              ...         ...        ...       ...   ...      ...   
39165             99          90         14   Charter  1635  1043130   
39166             95          70         14   Charter  1635  1043130   
39167             73          84         14   Charter  1635  1043130   
39168             99          90         14   Charter  1635  1043130   
39169             95          75         14   Charter  1635  1043130   

       passing_math  passing_reading  
4              True             True  
5              True             True  
6              True             True  
8              True             True  
9              True             True  
...             ...              ...  
39165          True             True  
39166          True             True  
39167          True             True  
39168          True             True  
39169          True             True  

[25528 rows x 13 columns]

In [23]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [24]:
school_data_complete

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  \
0                 66          79          0  District  2917  1910635   
1                 94          61          0  District  2917  1910635   
2                 90          60          0  District  2917  1910635   
3                 67          58          0  District  2917  1910635   
4                 97          84          0  District  2917  1910635   
...              ...         ...        ...       ...   ...      ...   
39165             99          90         14   Charter  1635  1043130   
39166             95          70         14   Charter  1635  1043130   
39167             73          84         14   Charter  1635  1043130   
39168             99          90         14   Charter  1635  1043130   
39169             95          75         14   Charter  1635  1043130   

       passing_math  passing_reading  
0              True            False  
1             False             True  
2             False             True  
3             False            False  
4              True             True  
...             ...              ...  
39165          True             True  
39166          True             True  
39167          True             True  
39168          True             True  
39169          True             True  

[39170 rows x 13 columns]

In [25]:
# Use groupby by school_name
school_group = school_data_complete.groupby(["school_name"]).mean()
school_group["Per Student Budget"] = school_group["budget"]/school_group["size"]
school_group["% Passing Math"] = round(school_group["passing_math"]*100,2)
school_group["% Passing Reading"] = round(school_group["passing_reading"]*100,2)
school_group["% Overall Passing Rate"] = round(((school_group["passing_math"] + school_group["passing_reading"])/2)*100,3)

In [26]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary_data={ "School Type" :school_types,
                         "Total Students" : per_school_counts,
                         "Total School Budget" : per_school_budget,
                         "Per Student Budget" : per_school_capita,
                         "Average Math Score" : per_school_math,
                         "Average Reading Score" : per_school_reading,
                         "% Passing Math" : per_school_passing_math,
                         "% Passing Reading" : per_school_passing_reading,
                         "% Overall Passing" : overall_passing_rate }
per_school_summary = pd.DataFrame(per_school_summary_data)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("{:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("{:,.2f}".format)

In [27]:
# Display the DataFrame
per_school_summary

School Type  Total Students Total School Budget  \
school_name                                                             
Bailey High School       District            4976        3,124,928.00   
Cabrera High School       Charter            1858        1,081,356.00   
Figueroa High School     District            2949        1,884,411.00   
Ford High School         District            2739        1,763,916.00   
Griffin High School       Charter            1468          917,500.00   
Hernandez High School    District            4635        3,022,020.00   
Holden High School        Charter             427          248,087.00   
Huang High School        District            2917        1,910,635.00   
Johnson High School      District            4761        3,094,650.00   
Pena High School          Charter             962          585,858.00   
Rodriguez High School    District            3999        2,547,363.00   
Shelton High School       Charter            1761        1,056,600.00   
Thomas High School        Charter            1635        1,043,130.00   
Wilson High School        Charter            2283        1,319,574.00   
Wright High School        Charter            1800        1,049,400.00   

                      Per Student Budget  Average Math Score  \
school_name                                                    
Bailey High School                628.00           78.985371   
Cabrera High School               582.00           78.985371   
Figueroa High School              639.00           78.985371   
Ford High School                  644.00           78.985371   
Griffin High School               625.00           78.985371   
Hernandez High School             652.00           78.985371   
Holden High School                581.00           78.985371   
Huang High School                 655.00           78.985371   
Johnson High School               650.00           78.985371   
Pena High School                  609.00           78.985371   
Rodriguez High School             637.00           78.985371   
Shelton High School               600.00           78.985371   
Thomas High School                638.00           78.985371   
Wilson High School                578.00           78.985371   
Wright High School                583.00           78.985371   

                       Average Reading Score  % Passing Math  \
school_name                                                    
Bailey High School                  81.87784       66.680064   
Cabrera High School                 81.87784       94.133477   
Figueroa High School                81.87784       65.988471   
Ford High School                    81.87784       68.309602   
Griffin High School                 81.87784       93.392371   
Hernandez High School               81.87784       66.752967   
Holden High School                  81.87784       92.505855   
Huang High School                   81.87784       65.683922   
Johnson High School                 81.87784       66.057551   
Pena High School                    81.87784       94.594595   
Rodriguez High School               81.87784       66.366592   
Shelton High School                 81.87784       93.867121   
Thomas High School                  81.87784       93.272171   
Wilson High School                  81.87784       93.867718   
Wright High School                  81.87784       93.333333   

                       % Passing Reading  % Overall Passing  
school_name                                                  
Bailey High School             81.933280          54.642283  
Cabrera High School            97.039828          91.334769  
Figueroa High School           80.739234          53.204476  
Ford High School               79.299014          54.289887  
Griffin High School            97.138965          90.599455  
Hernandez High School          80.862999          53.527508  
Holden High School             96.252927          89.227166  
Huang High School              81.316421    

## Highest-Performing Schools (by % Overall Passing)

In [28]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head()

School Type  Total Students Total School Budget  \
school_name                                                           
Cabrera High School     Charter            1858        1,081,356.00   
Thomas High School      Charter            1635        1,043,130.00   
Griffin High School     Charter            1468          917,500.00   
Wilson High School      Charter            2283        1,319,574.00   
Pena High School        Charter             962          585,858.00   

                    Per Student Budget  Average Math Score  \
school_name                                                  
Cabrera High School             582.00           78.985371   
Thomas High School              638.00           78.985371   
Griffin High School             625.00           78.985371   
Wilson High School              578.00           78.985371   
Pena High School                609.00           78.985371   

                     Average Reading Score  % Passing Math  % Passing Reading  \
school_name                                                                     
Cabrera High School               81.87784       94.133477          97.039828   
Thomas High School                81.87784       93.272171          97.308869   
Griffin High School               81.87784       93.392371          97.138965   
Wilson High School                81.87784       93.867718          96.539641   
Pena High School                  81.87784       94.594595          95.945946   

                     % Overall Passing  
school_name                             
Cabrera High School          91.334769  
Thomas High School           90.948012  
Griffin High School          90.599455  
Wilson High School           90.582567  
Pena High School             90.540541

## Bottom Performing Schools (By % Overall Passing)

In [29]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
#school_data_complete.sort_values("Student ID")
bottom_schools = per_school_summary.sort_values("% Overall Passing", ascending=True)
bottom_schools.head()

School Type  Total Students Total School Budget  \
school_name                                                             
Rodriguez High School    District            3999        2,547,363.00   
Figueroa High School     District            2949        1,884,411.00   
Huang High School        District            2917        1,910,635.00   
Hernandez High School    District            4635        3,022,020.00   
Johnson High School      District            4761        3,094,650.00   

                      Per Student Budget  Average Math Score  \
school_name                                                    
Rodriguez High School             637.00           78.985371   
Figueroa High School              639.00           78.985371   
Huang High School                 655.00           78.985371   
Hernandez High School             652.00           78.985371   
Johnson High School               650.00           78.985371   

                       Average Reading Score  % Passing Math  \
school_name                                                    
Rodriguez High School               81.87784       66.366592   
Figueroa High School                81.87784       65.988471   
Huang High School                   81.87784       65.683922   
Hernandez High School               81.87784       66.752967   
Johnson High School                 81.87784       66.057551   

                       % Passing Reading  % Overall Passing  
school_name                                                  
Rodriguez High School          80.220055          52.988247  
Figueroa High School           80.739234          53.204476  
Huang High School              81.316421          53.513884  
Hernandez High School          80.862999          53.527508  
Johnson High School            81.222432          53.539172

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]
ninth_graders

In [ ]:
# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()
ninth_graders_scores

In [ ]:
# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

In [ ]:
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_math_scores,
        "10th": tenth_grader_math_scores,
        "11th": eleventh_grader_math_scores,
        "12th": twelfth_grader_math_scores})

In [ ]:
# Minor data wrangling
math_scores_by_grade.index.name = None

In [ ]:
# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
ninth_graders

In [ ]:
ninth_graders_scores = school_data_complete.groupby(["school_name"]).mean()
ninth_graders_scores

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [ ]:
# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [ ]:
# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th grade" : ninth_graders_scores,
                                        "10th grade" : tenth_graders_scores,
                                        "11th grade" : eleventh_graders_scores,
                                        "12th grade" : twelfth_graders_scores})

#formatting the table
reading_scores_by_grade["9th grade"] = reading_scores_by_grade["9th grade"].map("{:,.2f}".format)
reading_scores_by_grade["10th grade"] = reading_scores_by_grade["10th grade"].map("{:,.2f}".format)
reading_scores_by_grade["11th grade"] = reading_scores_by_grade["11th grade"].map("{:,.2f}".format)
reading_scores_by_grade["12th grade"] = reading_scores_by_grade["12th grade"].map("{:,.2f}".format)

In [ ]:
# Minor data wrangling
reading_scores_by_grade.index.name = None

In [ ]:
# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
bin_labels = ["$0-585", "$585-630", "$630-645", "$645-680"]
    
spending_bins

In [ ]:
per_school_summary

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

school_spending_df

In [ ]:
# Convert single column to int dtype.
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].astype('float')
school_spending_df

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], bins=spending_bins, labels=bin_labels)

school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
        "Average Reading Score": spending_reading_scores,
        "% Passing Math": spending_passing_math,
        "% Passing Reading": spending_passing_reading,
        "% Overall Passing": overall_passing_spending})

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels, include_lowest=True)
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({"Average Math Score": size_math_scores,
        "Average Reading Score": size_reading_scores,
        "% Passing Math": size_passing_math,
        "% Passing Reading": size_passing_reading,
        "% Overall Passing": size_overall_passing})

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

In [ ]:
average_math_score_by_type = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
average_reading_score_by_type = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"]).mean()["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Average Math Score": average_math_score_by_type,
        "Average Reading Score": average_reading_score_by_type,
        "% Passing Math": average_percent_passing_math_by_type,
        "% Passing Reading": average_percent_passing_reading_by_type,
        "% Overall Passing": average_percent_overall_passing_by_type})

# Display results
type_summary